# setup

In [ ]:
import sys
from PIL import Image as PILImage
from pdf2image import convert_from_path
from IPython.display import display, Image, HTML
import os
import hashlib
import re
import math
import numpy as np
import lyapunov1
sys.path.insert(0, "/Users/nicknassuphis")
from specparser import specparser, expandspec
from rasterizer import raster

In [ ]:
def spec2fn(s):
    parts = s.split(':')
    head = re.sub(r'[^A-Za-z0-9]+', '_', parts[0])
    h = hashlib.sha1(s.encode()).hexdigest()[:10]
    return f"{head}_{h}.png"


In [ ]:
def markus_page(page:int = 209):
    pdf = "papers/mario_markus_book.pdf"
    out_png = f"mm_pages/mm_page_{page}.png"
    if not os.path.exists(out_png):
        pages = convert_from_path(pdf, dpi=300, first_page=page, last_page=page)
        pages[0].save(out_png, "PNG")
    return out_png


In [ ]:
def side_by_side(p1, p2, out_path=None, gap=20):
    im1 = PILImage.open(p1)
    im2 = PILImage.open(p2)

    # make them the same (capped) height
    h = 2000

    def resize_to_height(im, h):
        w = int(im.width * h / im.height)
        return im.resize((w, h), PILImage.LANCZOS)

    im1 = resize_to_height(im1, h)
    im2 = resize_to_height(im2, h)

    total_w = im1.width + gap + im2.width

    combo = PILImage.new("RGB", (total_w, h), (255, 255, 255))
    combo.paste(im1, (0, 0))
    combo.paste(im2, (im1.width + gap, 0))

    if out_path is None:
        os.makedirs("mm_side_by_side", exist_ok=True)
        b1 = os.path.splitext(os.path.basename(p1))[0]
        b2 = os.path.splitext(os.path.basename(p2))[0]
        out_path = f"mm_side_by_side/{b1}__{b2}.png"

    combo.save(out_path)
    return out_path


In [ ]:
def lyapunov_image(spec):
    out_png = f"mm_replicate/{spec2fn(spec)}"
    if not os.path.exists(out_png):
        expandspec.FUNCS["gap"]=lyapunov1.get_all_palettes
        expandspec.FUNCS["glp"]=lyapunov1.get_long_palettes
        expandspec.FUNCS["gtp"]=lyapunov1.get_tri_palettes
        specs = expandspec.expand_cartesian_lists(spec)
        if not specs:
            return None
        tiles: list[np.ndarray] = []
        titles: list[str] = []
        for i, spec in enumerate(specs, start=1):
            rgb = lyapunov1.spec2lyapunov(spec, pix=500)
            rgb = np.flipud(rgb)
            tiles.append(rgb)
            titles.append(spec)
        n = len(tiles)
        cols = max(1, int(round(math.sqrt(n))))
        raster.save_mosaic_png_rgb(
            tiles=tiles,
            titles=titles,
            cols=cols,
            gap=20,
            out_path=out_png,
            invert=False,
            footer_pad_lr_px=48,
            footer_dpi=300,
            thumbnail=None,
        )
    return out_png

In [ ]:

pair=side_by_side(markus_page(105),lyapunov_image(
'eq948:A2B2,'
'a:-1,b:2.25,c:0,d:2,'
'LL:4.4348:4.5432,UL:4.5546:4.663,LR:4.5522:4.4258,'
'rgb:mh_eq:1:firebrick:black:sunset'
 ))
display(Image(filename=pair))

In [ ]:
pair=side_by_side(markus_page(106),lyapunov_image(
'eq948:A2B2,a:0.6,b:1.5,c:0,d:2,ll:5.13:5.375,ul:5.433:5.678,lr:5.375:5.13,rgb:mh_eq:1:firebrick:black:sunset'
 ))
display(Image(filename=pair))

In [ ]:
pair=side_by_side(markus_page(107),lyapunov_image(
'eq948:A2B2,a:2.4,b:2,c:0,d:2,ll:0.796:0.966,ul:0.918:1.088,lr:0.966:0.796,rgb:mh_eq:1:firebrick:black:sunset' \
 ))
display(Image(filename=pair))